In [1]:
!pip install py4j pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=996c01ce762bbb8dd798db4ae6221c7ee98a92b2b2975da06b92e3bed14cc8e7
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('treeconsult').getOrCreate()
data = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/data/dog_food.csv',inferSchema=True,header=True)

In [4]:
data.head(2)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0),
 Row(A=5, B=6, C=12.0, D=7, Spoiled=1.0)]

In [7]:
###### Build a Random Forest classifier model 
from pyspark.ml.feature import VectorAssembler
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [8]:
assembler = VectorAssembler(inputCols=['A','B','C','D'],outputCol='features')
output = assembler.transform(data)

In [9]:
from pyspark.ml.classification import RandomForestClassifier
rfc = RandomForestClassifier(labelCol='Spoiled',featuresCol='features')
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [10]:
final_data = output.select('features','Spoiled')

In [11]:
final_data.columns

['features', 'Spoiled']

In [12]:
rfc_model = rfc.fit(final_data)

In [14]:
final_data.head(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [13]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0307, 1: 0.0284, 2: 0.9122, 3: 0.0287})